In [1]:
import json
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image

In [ ]:
def plot_image(img_path):
    img = Image.open(img_path)
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.imshow(img)
    plt.axis('off')

In [3]:
# Specify the path to the JSON file
file_path = 'data/SGSM/pre_conditions.json'

# Open the JSON file and load its contents
with open(file_path, 'r') as file:
    data = json.load(file)

In [ ]:
data[0]

In [7]:
# Vehicle within 10 meters, in the same lane and in front of (R1)
r1 = []
for d in data:
    k = list(d.keys())[0]
    for entity, relationships in d[k]["non_ego_entities"].items():
        pa = (relationships['lessThan4m'] or relationships['4to7m'] or relationships['7to10m'])
        pb = relationships['inFrontOf']
        pc = relationships['inEgoLane']
        if pa and pb and pc:
            r1.append(k)
            break
print(f"There are {len(r1)} images with ego lane controlled by yellow/red traffic light")

There are 222 images with ego lane controlled by yellow/red traffic light


In [8]:
# Ego lane controlled by yellow/red light (R2)
r2 = []
for d in data:
    k = list(d.keys())[0]
    pa = d[k]['egoLaneRed'] or d[k]['egoLaneYellow']
    if pa:
        r2.append(k)
print(f"There are {len(r2)} images with ego lane controlled by yellow/red traffic light")

There are 5068 images with ego lane controlled by yellow/red traffic light


In [ ]:
# Ego lane is controlled by a green light and no entity is in front and in the same lane and within 10 meters (R3)
r3 = []
for d in data:
    k = list(d.keys())[0]
    pa = d[k]['egoLaneGreen']
    for entity, relationships in d[k]["non_ego_entities"].items():
        pb = not relationships['inFrontOf']
        pc = not relationships['inEgoLane']
        pd = not (relationships['lessThan4m'] or relationships['4to7m'] or relationships['7to10m'])
        if pa and pb and pc and pd:
            r3.append(k)
            break
print(f"There are {len(r3)} images with ego lane controlled by a green traffic light and no entity is in front and in the same lane and within 10 meters")

There are 115 images with ego lane controlled by a green traffic light and no entity is in front and in the same lane and within 10 meters


In [ ]:
# # Ego lane is controlled by a green light and no entity is in front and in the same lane and within 10 meters (R3)
# r3 = []
# for d in data:
#     k = list(d.keys())[0]
#     pa = d[k]['egoLaneGreen']
#     append = True
#     for entity, relationships in d[k]["non_ego_entities"].items():
#         pb = relationships['inFrontOf']
#         pc = relationships['inEgoLane']
#         pd = not (relationships['lessThan4m'] or relationships['4to7m'] or relationships['7to10m'] or relationships['16to25m'])
#         if pb or pc or pd:
#             append = False
#             break
#     if pa and append:
#         r3.append(k)
    
# print(f"There are {len(r3)} images with ego lane controlled by a green traffic light and no entity is in front and in the same lane and within 10 meters")

There are 178 images with ego lane controlled by a green traffic light and no entity is in front and in the same lane and within 10 meters


In [96]:
# Ego vehicle in the rightmost lane and not in a junction (R4)
r4 = []
for d in data:
    k = list(d.keys())[0]
    pa = d[k]['egoInRightmost']
    pb = not d[k]['egoInJunction']
    if pa and pb:
        r4.append(k)
print(f"There are {len(r4)} images with ego vehicle in the rightmost lane and not in a junction")

There are 3130 images with ego vehicle in the rightmost lane and not in a junction


In [97]:
# Ego vehicle in the leftmost lane and not in a junction (R5)
r5 = []
for d in data:
    k = list(d.keys())[0]
    pa = d[k]['egoInLeftmost']
    pb = not d[k]['egoInJunction']
    if pa and pb:
        r5.append(k)
print(f"There are {len(r5)} images with ego vehicle in the leftmost lane and not in a junction")

There are 5347 images with ego vehicle in the leftmost lane and not in a junction


In [118]:
# Vehicle in the lane to the left and within 7 meters (R6)
r6 = []
for d in data:
    k = list(d.keys())[0]
    for entity, relationships in d[k]["non_ego_entities"].items():
        pa = relationships['inLaneToLeft']
        pb = relationships['inFrontOf']
        pc = relationships['4to7m']
        if pa and pb and pc:
            r6.append(k)
            break
print(f"There are {len(r6)} images with vehicle in the lane to the left and within 7 meters")

There are 147 images with vehicle in the lane to the left and within 7 meters


In [119]:
# Vehicle in the lane to the right and within 7 meters (R6)
r7 = []
for d in data:
    k = list(d.keys())[0]
    for entity, relationships in d[k]["non_ego_entities"].items():
        pa = relationships['inLaneToRight']
        pb = relationships['inFrontOf']
        pc = relationships['4to7m']
        if pa and pb and pc:
            r7.append(k)
            break
print(f"There are {len(r7)} images with vehicle in the lane to the right and within 7 meters")

There are 543 images with vehicle in the lane to the right and within 7 meters


In [6]:
def create_folder_with_sm_links(r_name, r_list, prompt):
    output_path = f'data/SGSM/all_generated_data/{r_name}
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for ip in r_list:
        routen = ip.split("RouteScenario_")[1].split("/")[0]
        iname = f"r{routen}_{ip.split('/')[-1]}"
        cp = ip.replace('/images/', '/captions/').replace(".jpg", ".jpg.text")
        cname = iname.replace(".jpg", ".txt")

        img_target_path = os.path.join(output_path, iname)
        os.symlink(ip, img_target_path)

        cap_target_path = os.path.join(output_path, cname)
        with open(cap_target_path, 'w') as file:
            file.write(prompt)

In [109]:
create_folder_with_sm_links("r1_data", r1, "C4RL4. Vehicle within 10 meters, in the same lane and in front of (R1)")

In [110]:
create_folder_with_sm_links("r2_data", r2, "C4RL4. Ego lane controlled by yellow/red traffic light (R2)")

In [24]:
# create_folder_with_sm_links("r3_data", r3, "C4RL4. Ego lane is controlled by a green traffic light and no entity is in front and in the same lane and within 10 meters (R3)")
create_folder_with_sm_links("r3_data_new", r3, "C4RL4. Ego lane is controlled by a green traffic light and no entity is in front and in the same lane and within 10 meters (R3)")

In [122]:
create_folder_with_sm_links("r4_data", r4, "C4RL4. Ego vehicle in the rightmost lane and not in a junction (R4)")

In [123]:
create_folder_with_sm_links("r5_data", r5, "C4RL4. Ego vehicle in the leftmost lane and not in a junction (R5)")

In [124]:
create_folder_with_sm_links("r6_data", r6, "C4RL4. Vehicle in the lane to the left and within 7 meters (R6)")

In [125]:
create_folder_with_sm_links("r7_data", r7, "C4RL4. Vehicle in the lane to the right and within 7 meters (R7)")

# Create data folders from Pandas df

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("data/SGSM/sgsm_features_dataset.csv")

In [31]:
r1 = list(df[(df['f1'] == 1)].images)
create_folder_with_sm_links("r1_data", r1, "C4RL4. Vehicle within 10 meters, in the same lane and in front of (R1)")
r2 = list(df[(df['f2'] == 1)].images)
create_folder_with_sm_links("r2_data", r2, "C4RL4. Ego lane controlled by yellow/red traffic light (R2)")
r3 = list(df[(df['f3a'] == 1) & (df['f3b'] == 1)].images)
create_folder_with_sm_links("r3_data", r3, "C4RL4. Ego lane is controlled by a green traffic light and no entity is in front and in the same lane and within 10 meters (R3)")
r4 = list(df[(df['f4a'] == 1) & (df['f4b'] == 1)].images)
create_folder_with_sm_links("r4_data", r4, "C4RL4. Ego vehicle in the rightmost lane and not in a junction (R4)")
r5 = list(df[(df['f5a'] == 1) & (df['f5b'] == 1)].images)
create_folder_with_sm_links("r5_data", r5, "C4RL4. Ego vehicle in the leftmost lane and not in a junction (R5)")
r6 = list(df[(df['f6'] == 1)].images)
create_folder_with_sm_links("r6_data", r6, "C4RL4. Vehicle in the lane to the left and within 7 meters (R6)")
r7 = list(df[(df['f7'] == 1)].images)
create_folder_with_sm_links("r7_data", r7, "C4RL4. Vehicle in the lane to the right and within 7 meters (R7)")